# Семинар: Предобработка текстового корпуса (Harry Potter Dataset)

---

## **Описание данных**

В вашем распоряжении — набор данных, состоящий из двух файлов:

1. **`harry_potter_clean.txt`** — корпус романов о Гарри Поттере в формате `.txt`, уже частично очищенный от спецсимволов.  
2. **`harry_potter_raw.txt`** — тот же текст, но сохранённый в «сыром» виде: содержит различные знаки пунктуации, кавычки, лишние пробелы, переводы строк и другие артефакты оригинального текста.  
   Для удобства каждое предложение в этом файле оканчивается символом вертикальной черты (`|`).

---

## **Цель задания**

Ваша задача — провести **предобработку текста** для последующего обучения моделей распределённых представлений слов (например, Word2Vec).

Результатом работы должен быть чистый, структурированный корпус, где каждое предложение готово к токенизации.

---

## **Требования к обработке**

1. **Удалить все лишние спецсимволы**, кроме:
   - точки (`.`),
   - вопросительного знака (`?`),
   - восклицательного знака (`!`).

   Все остальные знаки — кавычки, запятые, скобки, апострофы, многоточия, дефисы и т. д. — должны быть удалены.

2. **Удалить символы перевода строки** (`\n`) и **возврата каретки** (`\r`),  
   чтобы весь текст стал одной сплошной строкой.

3. **Удалить весь технический и вспомогательный текст**, например:
   - номера страниц,
   - заголовки глав и книг,
   - служебные вставки вида “CHAPTER ONE”, “Page 24”, “HARRY POTTER AND THE...”.

4. **Добавить пробелы перед и после всех разрешённых спецсимволов (`.`, `?`, `!`)**,  
   чтобы они воспринимались как отдельные токены при разбиении.

   Пример:  
   ```
   "He looked surprised!What happened?"  
   → "He looked surprised ! What happened ?"
   ```

5. **Исправить все ошибочные слова, где спецсимвол оказался внутри слова**, например:
   - `!harry` → `harry`
   - `ron?` → `ron`
   - `hermione!granger` → `hermione granger`

   То есть символы `.`, `?`, `!` могут находиться **только в конце предложения**.

6. **Сохранить результат** в новый файл `harry_potter_preprocessed.txt`,  
   где каждое предложение будет заканчиваться одним из символов `.`, `?`, `!`  
   и отделяться пробелом.

---

## **Дополнительно**

После завершения предобработки проверьте качество результата:
- количество предложений (примерно соответствует исходному);
- отсутствие мусорных токенов (например, `..`, `!!!`, `?.`);
- корректное разделение предложений (ни одно не склеено).

---

## **Формат сдачи**

Загрузите готовый файл `harry_potter_preprocessed.txt` и краткий отчёт (`.md` или `.txt`), где укажите:
- какие регулярные выражения вы использовали,
- какие типы ошибок встретились,
- примеры до/после очистки (по 3 примера).

---

## **Пример шаблона кода**


In [17]:
# Импорт библиотек
import re

# Загрузка исходного файла
with open('sample_data/HarryPotterTxTFile.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# 1. Удаляем символы перевода строки и возврата каретки
text = text.replace('\r', '').replace('\n', ' ')

# 2. Удаляем технический и вспомогательный текст (номера страниц, заголовки и т.д.)
# Удаляем номера страниц вида "Page 24", "page 13" и т.п.
text = re.sub(r'\b(?:[Pp]age\s*\d+|стр\.?\s*\d+)\b', '', text)
# Удаляем заголовки глав вида "CHAPTER ONE", "Chapter 1", "CHAPTER 1"
text = re.sub(r'\bCHAPTER\s+[A-Z]+(?:[A-Z\s]+)?\b', '', text, flags=re.IGNORECASE)
text = re.sub(r'\bChapter\s+\d+\b', '', text, flags=re.IGNORECASE)
# Удаляем служебные вставки типа "HARRY POTTER AND THE..."
text = re.sub(r'\bHARRY\s+POTTER\s+AND\s+THE\s+[A-Z]+\b', '', text, flags=re.IGNORECASE)

# 3. Удаляем все лишние спецсимволы, кроме . ? !
# Сначала заменяем многоточия на точку (сохраняя конец предложения)
text = re.sub(r'\.{3,}', '.', text)
# Удаляем все спецсимволы кроме . ? !
text = re.sub(r'[^\w\s\.\?\!]', '', text)

# 4. Исправляем ошибочные слова, где спецсимвол оказался внутри слова
# Удаляем . ? ! из середины слов
text = re.sub(r'(\w+)[\.\?\!](\w+)', r'\1 \2', text)
# Удаляем . ? ! в начале слов
text = re.sub(r'[\.\?\!](\w+)', r'\1', text)
# Удаляем . ? ! в конце слов (но сохраняем для концов предложений)
# Этот случай обработаем позже

# 5. Добавляем пробелы перед и после всех разрешённых спецсимволов
text = re.sub(r'([\.\?\!])', r' \1 ', text)

# 6. Удаляем лишние пробелы и нормализуем их
text = re.sub(r'\s+', ' ', text)

# 7. Разбиваем на предложения и обрабатываем каждое
sentences = []
current_sentence = []
words = text.split()

for word in words:
    current_sentence.append(word)
    # Если слово заканчивается на . ? !, заканчиваем предложение
    if word.endswith(('.', '?', '!')):
        sentence = ' '.join(current_sentence)
        sentences.append(sentence)
        current_sentence = []

# Если осталось незаконченное предложение
if current_sentence:
    sentence = ' '.join(current_sentence)
    if not sentence.endswith(('.', '?', '!')):
        sentence += '.'
    sentences.append(sentence)

# 8. Собираем текст обратно
text = ' '.join(sentences)

# 9. Финальная очистка: удаляем лишние точки, восклицательные знаки и т.д.
text = re.sub(r'([\.\?\!])\s+([\.\?\!])+', r'\1', text)  # Убираем дублированные знаки препинания
text = re.sub(r'([\.\?\!])\1+', r'\1', text)  # Убираем повторяющиеся знаки препинания
text = re.sub(r'\s+', ' ', text).strip()  # Убираем лишние пробелы

# Сохранение результата
with open('harry_potter_preprocessed.txt', 'w', encoding='utf-8') as f:
    f.write(text)

print('Предобработка завершена. Итоговый файл сохранён как harry_potter_preprocessed.txt')
print(f'Количество предложений: {len(sentences)}')

Предобработка завершена. Итоговый файл сохранён как harry_potter_preprocessed.txt
Количество предложений: 80201


In [ ]:
# Импорт библиотек
import re

# Загрузка исходного файла
with open('harry_potter_raw.txt', 'r', encoding='utf-8') as f:
    text = f.read()


# тут код предобработки

# Сохранение результата
with open('harry_potter_preprocessed.txt', 'w', encoding='utf-8') as f:
    f.write(text)

print('Предобработка завершена. Итоговый файл сохранён как harry_potter_preprocessed.txt')